In [1]:
import os
import numpy as np
import pandas as pd
from scipy.stats.stats import pearsonr   

In [2]:
students_et_filepath = '/home/shuang/projects/eqtm/data/students/overlapMatrixEt.csv'
bedtools_et_filepath = '/home/shuang/projects/development_eqtm/data/eqtmZscores/withExpressionTSSMethyCpgOverlapGene/2017-12-09-eQTLsFDR-et0_withExpressionTssMethyOverlap_withGeneOverlap.txt'


In [3]:
students = pd.read_csv(students_et_filepath,sep='\t',index_col=0)
bedtools = pd.read_csv(bedtools_et_filepath,sep=',',index_col=0)

In [4]:
students.head()

,H3K9me3_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H3K27me3_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H3K4me1_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H2A_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H3K36me3_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H3K4me3_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H3K27ac_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H3K4me2_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H4K91ac_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,DNase_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,...,H3K56ac_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H2AK5ac_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H2BK12ac_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H4K5ac_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H3K9me1_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H3K14ac_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H3T11ph_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H2BK20ac_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H2AK9ac_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_,H4K12ac_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_
cg00582671_ENSG00000188290,0.0,1.000000,0.574803,0.142105,0.0,0.023622,0.086614,0.19685,0.267717,0.102362,...,0.007874,0.251969,0.346457,0.070866,0.0,0.228346,0.150794,0.039370,0.111111,0.214286
cg00582671_ENSG00000078369,0.0,1.000000,0.574803,0.142105,0.0,0.023622,0.086614,0.19685,0.267717,0.102362,...,0.007874,0.251969,0.346457,0.070866,0.0,0.228346,0.150794,0.039370,0.111111,0.214286
cg12257384_ENSG00000188290,0.0,0.992126,1.000000,0.742105,0.0,1.000000,1.000000,1.00000,0.976378,1.000000,...,0.448819,0.748031,0.937008,0.803150,0.0,0.968504,0.801587,0.346457,0.476190,0.928571
cg20062691_ENSG00000187608,0.0,0.149606,1.000000,0.794737,0.0,1.000000,1.000000,1.00000,0.984252,1.000000,...,0.653543,0.826772,0.952756,0.897638,0.0,0.976378,0.865079,0.330709,0.555556,0.968254
cg03811829_ENSG00000187608,0.0,0.149606,1.000000,0.794737,0.0,1.000000,1.000000,1.00000,0.984252,1.000000,...,0.645669,0.826772,0.952756,0.889764,0.0,0.976378,0.865079,0.330709,0.555556,0.968254


In [5]:
# modify students matrix
students['SNPName'] = [col.split('_')[0] for col in students.index]
students_modified = students.copy()
for col in students.columns:
    if col == 'H2A_ImputedGapped_Roadmap_all_celltypes_westra_final_unique_':
        students_modified = students_modified.rename(index=str,columns={col:'H2A.Z'})
    students_modified = students_modified.rename(index=str,columns={col:col.split('_')[0]})

students_uniqueCpgs = students_modified.drop_duplicates(subset='SNPName').set_index('SNPName')
students_uniqueCpgs.shape

(4287, 32)

In [6]:
students_uniqueCpgs.head()

,H3K9me3,H3K27me3,H3K4me1,H2A.Z,H3K36me3,H3K4me3,H3K27ac,H3K4me2,H4K91ac,DNase,...,H3K56ac,H2AK5ac,H2BK12ac,H4K5ac,H3K9me1,H3K14ac,H3T11ph,H2BK20ac,H2AK9ac,H4K12ac
SNPName,,,,,,,,,,,,,,,,,,,,,
cg00582671,0.0,1.000000,0.574803,0.142105,0.0,0.023622,0.086614,0.19685,0.267717,0.102362,...,0.007874,0.251969,0.346457,0.070866,0.0,0.228346,0.150794,0.039370,0.111111,0.214286
cg12257384,0.0,0.992126,1.000000,0.742105,0.0,1.000000,1.000000,1.00000,0.976378,1.000000,...,0.448819,0.748031,0.937008,0.803150,0.0,0.968504,0.801587,0.346457,0.476190,0.928571
cg20062691,0.0,0.149606,1.000000,0.794737,0.0,1.000000,1.000000,1.00000,0.984252,1.000000,...,0.653543,0.826772,0.952756,0.897638,0.0,0.976378,0.865079,0.330709,0.555556,0.968254
cg03811829,0.0,0.149606,1.000000,0.794737,0.0,1.000000,1.000000,1.00000,0.984252,1.000000,...,0.645669,0.826772,0.952756,0.889764,0.0,0.976378,0.865079,0.330709,0.555556,0.968254
cg11211792,0.0,0.149606,1.000000,0.784211,0.0,1.000000,1.000000,1.00000,0.976378,1.000000,...,0.629921,0.826772,0.952756,0.881890,0.0,0.976378,0.857143,0.330709,0.523810,0.960317


In [7]:
bedtools.head()

,PValue,SNPName,SNPChr,SNPChrPos,ProbeName,ProbeChr,ProbeCenterChrPos,CisTrans,SNPType,AlleleAssessed,...,H2BK15acgene,H3K9me1gene,H3K4me2gene,H3K27me3gene,H3K79me1gene,H2AK9acgene,H3T11phgene,H4K8acgene,H4K91acgene,H3K56acgene
0,3.271670e-310,cg07772999,2,113993052,ENSG00000189223,2,113996843,cis,C/T,C,...,0.188976,0.000000,0.968504,0.866142,0.307087,0.314961,0.724409,0.803150,0.850394,0.354331
1,3.271670e-310,cg03085549,16,75150819,ENSG00000166816,16,75148213,cis,C/T,C,...,0.149606,0.007874,1.000000,0.385827,0.692913,0.393701,0.724409,0.771654,0.850394,0.283465
2,3.271670e-310,cg24429836,16,75150744,ENSG00000166816,16,75148213,cis,C/T,C,...,0.149606,0.007874,1.000000,0.385827,0.692913,0.393701,0.724409,0.771654,0.850394,0.283465
3,3.271670e-310,cg16540789,16,75150736,ENSG00000166816,16,75148213,cis,C/T,C,...,0.149606,0.007874,1.000000,0.385827,0.692913,0.393701,0.724409,0.771654,0.850394,0.283465
4,3.271670e-310,cg07320140,16,75150563,ENSG00000166816,16,75148213,cis,C/T,C,...,0.149606,0.007874,1.000000,0.385827,0.692913,0.393701,0.724409,0.771654,0.850394,0.283465


In [8]:
features = students_uniqueCpgs.columns
features_withname = list(features.values)
features_withname.append('SNPName')
bedtools_uniqueCpgs = bedtools[features_withname].drop_duplicates(subset='SNPName').set_index('SNPName')

In [15]:
for cpgname in students_uniqueCpgs.index:
    try:
        bedtool_ratio = bedtools_uniqueCpgs.loc[cpgname]
    except:
        print('Not in bedtools:',cpgname)
    students_ratio = students_uniqueCpgs.loc[cpgname]
    try:
        pr = pearsonr(bedtool_ratio.values,students_ratio.values)[0]
    except:
        pr = 1
        print('Two invalid scalars:')
        print(bedtool_ratio.values)
        print(students_ratio.values)
    if pr<0.95:
        print(pr,cpgname)

/home/shuang/.pyenv/versions/3.6.3/envs/py3/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


0.9468323814644037 cg13304609
0.9390414786213468 cg26791242
0.9496493169042338 cg11947712
0.9375305058144702 cg18533397
0.9375305058144702 cg00924017
0.9375305058144702 cg22221025
0.937577106955217 cg26668989
0.9366977024752492 cg21033965
0.9383567646056371 cg09938479
0.9383567646056371 cg26915952
0.9494680327219476 cg03440799
0.9436699451706954 cg02974928
0.9436699451706954 cg02431597
0.9046608657593143 cg17439800
0.9448376039693291 cg27526649
0.9217270648854028 cg17177536
0.9404467162628839 cg16060369
0.9383397958035863 cg14971320
0.9057663647530039 cg09611599
0.948146406237189 cg15681626
0.9498467347211444 cg05924780
0.9470011650264815 cg05044173
0.9443814679786169 cg04450876
0.9410471372584925 cg06809675
0.9410471372584925 cg18211706
0.9301305967140707 cg10088332
0.9301305967140707 cg20451566
0.9235311441167463 cg03758367
0.9110190071399312 cg20576002
0.8626050490735329 cg23058239
0.9301981334250427 cg04647790
0.9309698577028392 cg09258993
0.931054028999 cg06100461
0.931054028999 c

In [17]:
a = bedtools_uniqueCpgs.loc['cg23058239']
b = students_uniqueCpgs.loc['cg23058239']
print(a.values)
print(b.values)

[0.         0.00787402 0.17322835 0.24409449 0.         0.00787402
 0.04724409 0.06299213 0.05511811 0.09448819 0.03149606 0.03149606
 0.02362205 0.04724409 0.         0.00787402 0.00787402 0.00787402
 0.00787402 0.         0.03937008 0.04724409 0.02362205 0.01574803
 0.03937008 0.05511811 0.         0.03149606 0.03149606 0.03149606
 0.01574803 0.04724409]
[0.         0.00787402 0.17322835 0.09210526 0.         0.00787402
 0.04724409 0.06299213 0.05511811 0.09448819 0.03149606 0.03149606
 0.02362205 0.04724409 0.         0.00787402 0.00787402 0.00813008
 0.00787402 0.         0.03937008 0.04724409 0.02362205 0.01574803
 0.03937008 0.05511811 0.         0.03149606 0.03174603 0.03149606
 0.01587302 0.04761905]


In [19]:
bedtools_uniqueCpgs.columns[3]

'H2A.Z'